# Upgrading from MathProgBase.jl to MathOptInterface.jl

+ MathProgBase.jl has been [depreciated](https://github.com/JuliaOpt/MathProgBase.jl/pull/231) since Jan 2019. It is therefore not officially supported by julia v1.2 and above. We should use [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl) instead. 
+ Other packages such as BinaryProvider.jl and Ipopt.jl needs to also be updated to the latest version or depreciated and use the latest one supported by the Julia community.
+ To ensure a smooth update, checkout the [official documentation](https://jump.dev/MathOptInterface.jl/dev/tutorials/mathprogbase/) as well as examples by [WiSER.jl](https://github.com/OpenMendel/WiSER.jl/blob/master/src/fit.jl) and [OrdinalMultinomialModels.jl](https://github.com/OpenMendel/OrdinalMultinomialModels.jl/blob/master/src/ordmnfit.jl)

In [16]:
using Revise
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface

ENV["COLUMNS"] = 240
BLAS.set_num_threads(1)
Threads.nthreads()

k = 0 # number of causal SNPs

qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2024,
    y_distribution = Bernoulli,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

@show qc_model;

@time optm = QuasiCopula.fit!(qc_model);

qc_model = Quasi-Copula Variance Component Model
  * base distribution: Bernoulli
  * link function: LogitLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 15

gcm.β = [0.4243775487646154, -0.21124836575652164, 0.2051203650071014, 0.18955742133785242, 0.2000674086118032, -0.19946857554782652, 0.20813046939769364, -0.1959624974340227, 0.18888295619966955, -0.1895858430850509, -0.20967942115370897, -0.18902276604617343, 0.19592889680296988, -0.20589983107141385, -0.19223146684017767]
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       17
                     variables with only lower bounds:        2
                variables with lower 

In [2]:
T = Float64
const MOI = MathOptInterface
solver = Ipopt.Optimizer()
solver_config = Dict("print_level"           => 5, 
                 "mehrotra_algorithm"    => "yes",
                 "warm_start_init_point" => "yes",
                 "max_iter"              => 100)
solvertype = typeof(solver)
QuasiCopula.config_solver(solver, solver_config)
gcm = qc_model

    solvertype = typeof(solver)
    solvertype <: Ipopt.Optimizer ||
        @warn("Optimizer object is $solvertype, `solver_config` may need to be defined.")
    config_solver(solver, solver_config)

    # set lower bounds and upper bounds of parameters
    npar = gcm.p + gcm.m
    lb   = fill(-Inf, npar)
    ub   = fill(Inf, npar)
    offset = gcm.p + 1
    for k in 1:gcm.m
        lb[offset] = 0
        offset += 1
    end

    # set up NLP optimization problem
    MOI.empty!(solver)
    NLPBlock = MOI.NLPBlockData(
        MOI.NLPBoundsPair.(lb, ub), gcm, true
    )
    par0 = Vector{T}(undef, npar)
    solver_pars = MOI.add_variables(solver, npar)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)

    # initial condition
    QuasiCopula.initialize_model!(gcm)
    QuasiCopula.modelpar_to_optimpar!(par0, gcm)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end

    MOI.optimize!(solver)


LoadError: UndefVarError: `config_solver` not defined

## TrajGWAS.jl test

In [ ]:
using CSV, DataFrames, WiSER
filepath = normpath(joinpath(dirname(pathof(WiSER)), "../data/"))
df = DataFrame(CSV.File(filepath * "sbp.csv"))

m = WSVarLmmModel(
    @formula(sbp ~ 1 + agegroup + gender + bmi_std + meds), 
    @formula(sbp ~ 1 + bmi_std), 
    @formula(sbp ~ 1 + agegroup + meds + bmi_std),
    :id, df);

solver = Ipopt.Optimizer()


    solvertype = typeof(solver)
    solvertype <: Ipopt.Optimizer ||
        @warn("Optimizer object is $solvertype, `solver_config` may need to be defined.")
    # Pass options to solver
    WiSER.config_solver(solver, solver_config)
    # set up NLP optimization problem
    npar = m.l + ◺(m.q)
    MOI.empty!(solver)
    lb   = T[]
    ub   = T[]

    NLPBlock = MOI.NLPBlockData(
        MOI.NLPBoundsPair.(lb, ub), m, true
    )

    par0 = Vector{T}(undef, npar)
    WiSER.modelpar_to_optimpar!(par0, m)

    solver_pars = MOI.add_variables(solver, npar)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end

    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
